In [1764]:
FORM_MAPS = {
    "le génitif": "genitive",
    "gen.": "genitive",
    "gén.": "genitive",
    "gén": "genitive",
    "dat.": "dative",
    "dat": "dative",
    "plur.": "plural",
    "plur": "plural",
    "futur": "future",
    "fut.": "future",
    "participe": "participle",
    "comp.": "comparative",
    "comp": "comparative",
    "compar.": "comparative",
    "compar": "comparative",
    "impér.": "imperative",
    "impér": "imperative",
    "diminutif": "diminutive",
    "vocatif": "vocative",
    "gén. fém.": "genitive_feminine",
    "superl.": "superlative"
}

In [2082]:
import json
with open("sjoestedt_phonetique.json") as inf:
    DATA = json.load(inf)

In [424]:
section = 30

if not section in DATA:
    DATA[section] = []

In [384]:
DATA[section] = []

In [1765]:
import re
REGEX_CASE = rf"^([^(]+) \(([^)]+)\) « ([^»]+) », ({'|'.join(FORM_MAPS.keys())}) ([^(]+) \(([^)]+)\)$"
REGEX_CASE_OF = rf"^([^(]+) \(([^)]+)\),? ({'|'.join(FORM_MAPS.keys())}) de ([^«]+) « ([^»]+) »$"
REGEX_CASE_OF_BOTH = rf"^([^(]+) \(([^)]+)\),? ({'|'.join(FORM_MAPS.keys())}) de ([^«]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_BASE = r"^([^(]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_JUST_FRENCH = r"^([^(]+) « ([^»]+) »$"
REGEX_FROM = r"^([^(]+) \(([^)]+)\) « ([^»]+) »,? de ([^(]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_IN = r"^([^(]+) \(([^)]+)\), dans ([^«]+) « ([^»]+) »$"
REGEX_OR = r"^([^(]+) \(([^)]+)\) ou ([^(]+) \(([^)]+)\) « ([^»]+) »$"

def extract(text, splitter, counter):
    text = text.replace("\u00ad", "").replace("\n", " ")
    for p in text.split(splitter):
            m = re.match(REGEX_BASE, p.strip())
            mjf = re.match(REGEX_JUST_FRENCH, p.strip())
            mf = re.match(REGEX_FROM, p.strip())
            min = re.match(REGEX_IN, p.strip())
            mc = re.match(REGEX_CASE, p.strip())
            mcofb = re.match(REGEX_CASE_OF_BOTH, p.strip())
            mcof = re.match(REGEX_CASE_OF, p.strip())
            mor = re.match(REGEX_OR, p.strip())
            if m:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": m.group(1),
                    "irish": m.group(2),
                    "french": m.group(3),
                })
                counter += 1
            elif mjf:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mjf.group(1),
                    "french": mjf.group(2),
                })
                counter += 1
            elif mf:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mf.group(1),
                    "irish": mf.group(2),
                    "french": mf.group(3).replace("\xad", ""),
                    "from": mf.group(5)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mf.group(4),
                    "irish": mf.group(5),
                    "french": mf.group(6).replace("\xad", ""),
                })
                counter += 1
            elif mor:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mor.group(1),
                    "irish": mor.group(2),
                    "french": mor.group(5),
                    "alt": mor.group(4)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mor.group(3),
                    "irish": mor.group(4),
                    "french": mor.group(5),
                    "alt": mor.group(2)
                })
                counter += 1
            elif min:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": min.group(1),
                    "irish": min.group(2),
                    "in": min.group(3)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": min.group(3),
                    "french": min.group(4),
                })
                counter += 1
            elif mc:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mc.group(1),
                    "irish": mc.group(2),
                    "french": mc.group(3),
                    FORM_MAPS[mc.group(4)]: mc.group(6)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mc.group(5),
                    "irish": mc.group(6),
                    f"{FORM_MAPS[mc.group(4)]}_of": mc.group(2)
                })
                counter += 1
            elif mcofb:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcofb.group(1),
                    "irish": mcofb.group(2),
                    f"{FORM_MAPS[mcofb.group(3)]}_of": mcofb.group(5)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "irish": mcofb.group(5),
                    "transcription": mcofb.group(4),
                    "french": mcofb.group(6),
                    FORM_MAPS[mcofb.group(3)]: mcofb.group(2)
                })
                counter += 1
            elif mcof:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcof.group(1),
                    "irish": mcof.group(2),
                    f"{FORM_MAPS[mcof.group(3)]}_of": mcof.group(4)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcof.group(4),
                    "french": mcof.group(5),
                    FORM_MAPS[mcof.group(3)]: mcof.group(2)
                })
                counter += 1
            else:
                print(p)
    print(counter)


In [2138]:
REST = {
    275: "ˈʃαnˌdɪnʹɩ (seanduine) « vieillard » ; ˈʃαnəˌvʹαn (seana­bhean) « vieille femme » ; ˌʃαnəˈvro:g (seana-bhróg) « vieux soulier »",
    272: "trɑ:nˈho:ⁱnʹˌtʹi: (tráthnóintí) « soirées »",
    273: "ˌstrɑpəˈdo:ⁱrʹəχt (strapadóireacht) « escalade » ; kʷɩˌdʹαχtəˈnᴜ:ⁱlʹ (cuideacht­anamhail) « sociable » ; χəmʲɑ:dɩˌdʹi:ʃtʹ (choimeádaidís) « ils avaient coutume de garder » ; ˈmɑ:hɩrʹᵊχə (máithreacha), plur. de mɑ:hɩrʹ (máthair) « mère » ; ˈfɔhərəgə (foth­ragadh) « hâte, affole­ment »",
    274: "ˌstrapəˈdo:ⁱrʹəχtə (strapadóireachta), gén. de ˌstrɑpəˈdo:ⁱrʹəχt (strapadóireacht) « escalade »"
}

, mais ˌʃαnəˈvro:g (seana-bhróg) « vieux soulier », d’où « histoire rebattue ».

ˈlʹαˌχᴜmə (leathchuma) « partialité », mais ˌlʹαˈʃg̬ʹì:al (leath­sgéal) « excuse ».

§ 276. Premier terme comportant une voyelle longue :

ˈmʹi:ˌvlɑs (mí-bhlas) « mauvais goût » ; ˌmʹi·ˈǥlʹì:as (mí-ghléas) « mauvais état de fonc­tionne­ment» ; ˌmʹi·ˈnɑ:ⁱrʹəχ (mí-náireach) « éhonté » ; ˌdʹi·ˈkʹe:lʹɩ (díthcéille) « folie » ; ˌkwᴇ̈:ᵊχrᴜ·ə (caoch-ruadh) « maladie des pommes de terre » ; ˌlᴀᴜmˈᴇ̈:stə (lom-aosta) « d’un certain âge » ; ˌᴀᴜnˈro: (annró) « cata­strophe » ; ˌro:ˈᴇ̈:stə (ro-aosta) « trop âgé » ; mais : ˈrɪ:ˌvah (rí-mhaith) « excellent » ; ˈkʷᴇ̈:ᵊlˌχʷɩdʹ (caol-chuid) « toute petite partie » ; ˈdʹi·ənˌvah (dian-mhaith) « excellent » ; ˈlᴀᴜmˌçαrt (lom-cheart) « droit strict » ; ˈro:ˌjαl (ro-gheal) « trop brillant » ; ˈᴀᴜnˌçαrt (anncheart) «injustice ».

§ 277. Composés formés de trois termes :

ˌbᴜnöʃˈg̬ʲᴜ:n (bun ós cionn) « cul par-dessus tête » ; ˌdʌləvᵊˈla·χ (dul-í-bhfolach) « cache-cache » ; ˌkʹimʹɩləˈvɑ:ⁱlʹi·nʹ (címil an mháilín) « tiraille­ment, tracasse­ment »

In [2139]:
# DATA[section] = []
section = 275
if not section in DATA:
    DATA[section] = []
text = REST[section].replace(" : ", " ; ")
text = text.replace("\u00ad", "")
text = text.replace("»", " »")
text = text.replace("  »", " »")
text = text.replace(", mais", " ; ")
extract(text, ";", 10)

13


In [2134]:
def split_trans(pos):
    if " ou " in DATA[section][pos]["transcription"]:
        DATA[section][pos]["transcription"] = DATA[section][pos]["transcription"].split(" ou ")
    elif " et " in DATA[section][pos]["transcription"]:
        DATA[section][pos]["transcription"] = DATA[section][pos]["transcription"].split(" et ")
    elif " à côté de " in DATA[section][pos]["transcription"]:
        DATA[section][pos]["transcription"] = DATA[section][pos]["transcription"].split(" à côté de ")

for s in range(len(DATA[section])):
    if "transcription" in DATA[section][s]:
        split_trans(s)

In [2137]:
import json

if str(section) in DATA and section in DATA:
    tmp = DATA[str(section)] + DATA[section]
    DATA[str(section)] = tmp
    del(DATA[section])

with open("sjoestedt_phonetique.json", "w") as outf:
    json.dump(DATA, outf)

In [2136]:
def french_to_note(pos):
    DATA[section][pos]["french_note"] = DATA[section][pos]["french"]
    del(DATA[section][pos]["french"])

def contrast(pos, pos1):
    if 'irish' in DATA[section][pos1]:
        DATA[section][pos]['contrast'] = DATA[section][pos1]['irish']
    else:
        DATA[section][pos]['contrast'] = DATA[section][pos1]['transcription']
    DATA[section][pos]['contrast_id'] = DATA[section][pos1]['id']

def contrast_next(pos):
    contrast(pos, pos+1)

def syllabified(item):
    if type(item["transcription"]) == str:
        if "-" in item["transcription"]:
            item["syllabified"] = item["transcription"]
            item["transcription"] = item["transcription"].replace("-", "")
    else:
        if "-" in item["transcription"][0]:
            item["syllabified"] = item["transcription"]
            item["transcription"] = [x.replace("-", "") for x in item["transcription"]]

def syllabify():
    for s in DATA[section]:
        syllabified(s)

# syllabify()
# contrast(8, 10)
# DATA[264][0]["transcription"] = DATA[264][0]["transcription"].split(", ")
# contrast_next(-3)
# del(DATA[section][-1])
# del(DATA[section][-2]['contrast'])
# del(DATA[section][-2]['contrast_id'])
# DATA[section].append(
#     {
#       "section": 254,
#       "id": "254_4",
#       "transcription": "to:rhnʲəχ",
#       "syllabified": "to:r-hnʲəχ"
#     }
# )
# DATA[section][0]["syllabified_incorrect"] = "nʹαvʹ-rʹɑ:"
# DATA[section][-1]["french_note"] = "litt. « aux pieds secs »"
# del(DATA[section][0]["french_note"])
# DATA[section][0]["cf_section"] = 215

# del(DATA[section][-2]["french"])
contrast_next(-2)
DATA[section][-2]['french_note'] = "litt. « l’aube rouge »"
DATA[section][-1]['french_note'] = "litt. « honte rouge »"

In [1502]:
mods = [
    {
      "section": 17,
      "id": "17_23",
      "irish": "lomaim",
      "transcription": "lɔmʷɩmʹ",
      "french": "je me dépouille, je deviens chauve",
      "future": "lomfad"
    },
    {
      "section": 177,
      "id": "177_25",
      "transcription": "sɔnə",
      "irish": "sona",
      "french": "heureux"
    },
    {
      "section": 177,
      "id": "177_23",
      "transcription": "krɔmʷɩmʹ",
      "irish": "cromaim",
      "french": "je courbe"
    },
   {
      "section": 177,
      "id": "177_24",
      "transcription": "fɔnəvər",
      "irish": "fonnmhar",
      "french": "désireux"
    },
]
modwith = ["lo̤mʷɩmʹ", "so̤nə", "kro̤mʷɩmʹ", "fo̤nəvər"]

section = 180
DATA[section] = []
for i in range(len(mods)):
    mod = mods[i]
    mod["section"] = section
    mod["compare_id"] = mod["id"]
    mod["id"] = f"{section}_{i + 1}"
    mod["transcription"] = modwith[i]
    DATA[section].append(mod)


In [2083]:
START = DATA["261"][0:7]
REST = DATA["261"][7:]

def increment_id(item):
    itemid = item["id"]
    p = itemid.split("_")
    item["id"] = f'{item["section"]}_{int(p[1])+1}'
    return item

ADD = {
      "section": 261,
      "id": "261_8",
      "transcription": "məˈlaχtu:",
      "cf_section": 268
}

DATA["261"] = START + [ADD] + [increment_id(x) for x in REST]